# Data Lake on S3

In [1]:
from pyspark.sql import SparkSession

#import databricks.koalas as ks
import os
import configparser


from datetime import datetime

from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format


# Make sure that your AWS credentials are loaded as env vars

In [2]:
config = configparser.ConfigParser()

#Normally this file should be in ~/.aws/credentials
config.read('dl.cfg')

os.environ["AWS_ACCESS_KEY_ID"]= config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"]= config['AWS']['AWS_SECRET_ACCESS_KEY']

# Create spark session with hadoop-aws package

In [3]:
spark = SparkSession.builder\
                    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

# Load data from S3

In [4]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://project4dend/"
    

In [10]:
#kdf = ks.read_json("data/song_data/*/*/*/*.json")# s3a://udacity-dend/song_data/*/*/*/*.json to read the whole data

# get filepath to song data file
#song_data = input_data + "data/song_data/A/B/C/*.json"
    
    # read song data file
df = spark.read.json("data/song_data/A/B/C/*.json")
   

In [26]:
#kdf.dtypes
df.head()
 # extract columns to create songs table
songs_table = df.select("song_id","title","artist_id","year","duration").drop_duplicates()
songs_table

DataFrame[song_id: string, title: string, artist_id: string, year: bigint, duration: double]

In [33]:

songs_table.write.parquet("songs/", mode="overwrite", partitionBy=["year","artist_id"])
 
    # extract columns to create artists table
artists_table = df.select("artist_id","artist_name","artist_location","artist_latitude","artist_longitude").drop_duplicates()
    
    # write artists table to parquet files
artists_table.write.parquet("artists/", mode="overwrite")
    
 


In [32]:

artists_table


DataFrame[artist_id: string, artist_name: string, artist_location: string, artist_latitude: double, artist_longitude: double]

# saving the data

In [16]:


songs_table = (ks.sql('''
               SELECT 
               DISTINCT
               row_number() over (ORDER BY year,title,artist_id) id,
               title,
               artist_id,
               year,
               duration
               FROM 
                   {kdf}''')
              )


songs_table

,id,title,artist_id,year,duration
0,1,A Whiter Shade Of Pale (Live @ Fillmore West),ARLTWXK1187FB5A3F8,0,326.00771
1,2,Amor De Cabaret,ARKRRTF1187B9984DA,0,177.47546
2,3,Auguri Cha Cha,ARHHO3O1187B989413,0,191.84281
3,4,Baby Come To Me,AR9AWNF1187B9AB0B4,0,236.93016
4,5,Broken-Down Merry-Go-Round,ARQGYP71187FB44566,0,151.84934
5,6,Clementina Santafè,ARGCY1Y1187B9A4FA5,0,153.33832
6,7,Den Pai Alo,AROGWRA122988FEE45,0,243.82649
7,8,Der Kleine Dompfaff,ARJIE2Y1187B994AB7,0,152.92036
8,9,Drop of Rain,AR10USD1187B99F3F1,0,189.57016
9,10,Erica (2005 Digital Remaster),AREDBBQ1187B98AFF5,0,138.63138


In [17]:
songs_table.shape

(71, 5)

### Apache Parquet Introduction

Apache Parquet is a columnar file format that provides optimizations to speed up queries and is a far more efficient file format than CSV or JSON, supported by many data processing systems.




### Spark Write DataFrame to Parquet file format

Using spark.write.parquet() function we can write Spark DataFrame to Parquet file.




### Spark parquet partition – Improving performance

> Partitioning is a feature of many databases and data processing frameworks and it is key to make jobs work at scale. We can do a parquet file partition using spark partitionBy function.

> Parquet Partition creates a folder hierarchy for each spark partition; we have mentioned the first partition as year followed by artist_id hence, it creates a artist_id folder inside the year folder.

In [18]:
    
(songs_table
 .to_spark()
 .write
 .partitionBy("year", "artist_id")
 .parquet('songs/')
)

## Deployment

1. Install `awscli`

2. run `aws configure` 
    * AWS Access Key ID : 
    * AWS Secret Access Key : 
    * Default region name: `us-west-2`
    * Default output format : `json`
    
3. **copy all the necessary files to an s3 bucket**

    * Ex: `aws s3 cp <filename> s3://<bucket_name>`


4. **Run EMR create script with the etl job**

```
aws emr create-cluster --name "Spark cluster with step" \
    --release-label emr-5.30.1 \
    --applications Name=Spark \
    --log-uri s3://dendsparktutorial/logs/ \
    --ec2-attributes KeyName=emr-key \
    --instance-type m5.xlarge \
    --instance-count 3 \
    --bootstrap-actions Path=s3://dendsparktutorial/emr_bootstrap.sh \
    --steps Type=Spark,Name="Spark program",ActionOnFailure=CONTINUE,Args=[--deploy-mode,cluster,--master,yarn,s3://dendsparktutorial/src/koalas_etl.py] \
    --use-default-roles \
    --auto-terminate
```